In [1]:
import pickle
import gzip
import numpy as np
from numpy.random import seed
import sys
import h5py
import time 
from math import floor, sqrt, ceil
from sklearn.preprocessing import MinMaxScaler
from itertools import chain

In [72]:
from NN_no_hidden import NN as NN1

In [71]:
def make_structured_input_for_root_NN(bin_data, labels, split, dim_set):
    position_labels = np.copy(labels)
    size_batch = dim_set // split
    remain = dim_set % split #ceil((dim_set / split - size_batch) * split)
    for i in range(0, split, 1):
        position_labels[(i*size_batch):(((i+1)*size_batch if i < (split-1) else (i+1)*size_batch+remain))]=i
    
    position_labels = np.reshape(position_labels, (-1,1))
    splitted_labels = [np.reshape(labels[(i*size_batch):(((i+1)*size_batch if i < (split-1) else (i+1)*size_batch+remain))], (-1,1)) for i in range(split)]
    splitted_bin_data = [bin_data[(i*size_batch):(((i+1)*size_batch if i < (split-1) else (i+1)*size_batch+remain))] for i in range(split)] 
    
    perm_splitted_bin_data = []
    perm_splitted_labels = []
    for i in range(split):
        #np.random.RandomState(42)
        p_split = np.random.RandomState(seed=0).permutation(len(splitted_bin_data[i]))    
        perm_splitted_labels.append(splitted_labels[i][p_split])
        perm_splitted_bin_data.append(splitted_bin_data[i][p_split])
    

In [73]:
split = 1
with h5py.File('Resource2/file3uniform_bin.sorted.mat','r') as f:
    data = f.get('Sb') 
    bin_data = np.array(data, dtype=np.bool)
    bin_data = np.transpose(bin_data)
    bin_data = np.flip(bin_data,axis=1)
    dim_set = len(bin_data)
labels = np.linspace(1, len(bin_data), num=len(bin_data), dtype=np.int64)
bin_data, splitted_bin_data, position_labels, splitted_labels = make_structured_input_for_root_NN(bin_data, labels, split, dim_set)

TypeError: cannot unpack non-iterable NoneType object

In [55]:
i = 0
while True:
    if not np.any(bin_data[:,i]):
        i += 1
    else:
        print(i)
        break;

33


In [57]:
np.any(bin_data[:,63])

True

In [58]:
x = bin_data[:,i:bin_data.shape[1]]

In [59]:
x.shape

(512, 31)

In [60]:
np.any(x)

True

In [78]:
np.any(bin_data[:,0:64])

True

In [66]:
N_FEATURES = 64-33
N_CLASSES = 1

In [67]:
weights = np.random.RandomState(seed=0).normal(loc=0., scale = 0.05 ,size=(N_FEATURES, N_CLASSES)).astype(np.float32)
bias = np.random.RandomState(seed=0).normal(loc=0., scale = 0.05 ,size=(1, N_CLASSES)).astype(np.float32)
w= [[weights, bias]]

In [68]:
nn = NN1.NN(training=[bin_data, splitted_labels], testing=[[0],[0]], lr=1e-3, mu=0.9, output_classes=1, lambd=0, minibatch=64, disableLog=True)
nn.addLayers(['leakyrelu'], ww)
nn.set_patience(10)
loss = nn.train(stop_function=3, num_epochs=20000)

SyntaxError: invalid syntax (<ipython-input-68-520444b7c293>, line 1)

In [ ]:
predict = nn.predict(splitted_bin_data[s])
pr = np.ceil((np.multiply(predict,predict>0)))
lab = splitted_labels[s]
max_err = np.max(np.abs(pr-lab)).astype("int64")
max_errs.append(max_err)